In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
from matplotlib import pyplot as plt
import seaborn as sns
rc('font', family = 'AppleGothic')

In [2]:
train = pd.read_csv("EDA/data/train.csv")
data = pd.read_csv('external_data.csv',header='infer')

## 0. 전처리

- 세대수 = 총세대수
- 기본임대보증금 = 임대보증금
- 주택유형명 = 임대건물구분
- 공급유형명 = 공급유형
- 광역도시명 = 지역
- 공급전용면적 = 전용면적
- 주차수 = 단지내주차면수(완전히 동일하지는 않음!!)

In [3]:
train['임대료'] = train['임대료'].replace('-',np.nan)
train['임대보증금'] = train['임대보증금'].replace('-',np.nan)
train['임대료'] = train['임대료'].apply(lambda x : float(x))
train['임대보증금'] = train['임대보증금'].apply(lambda x : float(x))

In [4]:
data = data.rename(columns = {'세대수':'총세대수','기본임대보증금':'임대보증금','주택유형명':'임대건물구분','공급유형명':'공급유형','광역시도명':'지역','공급전용면적':'전용면적','주차수':'단지내주차면수'})

In [5]:
print(sorted(train['지역'].unique()))
print(sorted(data['지역'].unique()))

['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']
['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']


In [6]:
print(sorted(data['공급유형'].unique()))
print(sorted(train['공급유형'].unique()))

['10년임대', '50년임대', '5년임대', '국민임대', '매입임대', '영구임대', '장기전세', '행복주택']
['공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)', '공공임대(분납)', '국민임대', '영구임대', '임대상가', '장기전세', '행복주택']


In [7]:
data.loc[data.공급유형.isin(['10년임대']), '공급유형'] = '공공임대(10년)'
data.loc[data.공급유형.isin(['50년임대']), '공급유형'] = '공공임대(50년)'
data.loc[data.공급유형.isin(['5년임대']), '공급유형'] = '공공임대(5년)'

In [8]:
data1 = data[data['임대건물구분']=='아파트']
train1 = train[train['임대건물구분']=='아파트']

In [9]:
data1 = data1[['지역','총세대수','전용면적','단지명','공급유형','임대건물구분','임대보증금','단지내주차면수']]
train1 = train1[['단지코드','총세대수','지역','전용면적','임대보증금','공급유형','임대건물구분','임대보증금','단지내주차면수']]

## 기준1 : 지역, 공급유형, 총세대수

In [10]:
result = pd.merge(train1,data1,how='left',on=['지역','공급유형','총세대수'])[['단지코드','단지명']].drop_duplicates()

In [11]:
print('NA 개수: ',result['단지명'].isna().sum())
print('특정된 단지코드 수: ',train['단지코드'].nunique()-result['단지명'].isna().sum())
print('전체 단지코드 수: ',train['단지코드'].nunique())
print('NA 비율: ', round(result['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  94
특정된 단지코드 수:  329
전체 단지코드 수:  423
NA 비율:  0.22


In [12]:
result.head()

,단지코드,단지명
0,C2483,포항장량휴먼시아1단지
32,C2515,미수휴먼시아
60,C1407,도안 휴먼시아 4단지
137,C1945,호평마을 주공휴먼시아
155,C1470,장신휴먼시아1단지


## 기준 2 : 지역, 공급유형, 전용면적, 임대보증금

In [156]:
l = result[result['단지명'].isna()==True]['단지코드'].values
na = []
for i in l:
    if result[result['단지코드']==i].count()['단지코드'] == 1:
        na

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [13]:
datana = result[result['단지명'].isna()==True]['단지코드'].values
n = len(datana)

In [14]:
datana = train.loc[train['단지코드'].isin(datana)]
datana = datana[datana['임대건물구분']=='아파트']

In [15]:
result2 = pd.merge(datana,data1,how='left',on=['지역','공급유형','전용면적','임대보증금'])[['단지코드','단지명']].drop_duplicates()

In [16]:
result2.head()

,단지코드,단지명
0,C1925,입암주공3단지아파트
2,C1874,쌍용주공7단지1차아파트
4,C1874,공주옥룡주공1차아파트
6,C1874,읍내동 주공아파트
17,C2416,진주평거 주공2단지


In [17]:
print('NA 개수: ',result2['단지명'].isna().sum())
print('특정된 단지코드 수: ',n-result2['단지명'].isna().sum())
print('전체 단지코드 수: ',n)
print('NA 비율: ', round(result2['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  39
특정된 단지코드 수:  55
전체 단지코드 수:  94
NA 비율:  0.09


## 기준3 : 지역, 공급유형, 전용면적, 단지내주차면수

In [18]:
datana = result2[result2['단지명'].isna()==True]['단지코드'].values
n = len(datana)

In [19]:
datana = train.loc[train['단지코드'].isin(datana)]
datana = datana[datana['임대건물구분']=='아파트']

In [20]:
result3 = pd.merge(datana,data1,how='left',on=['지역','공급유형','전용면적','단지내주차면수'])[['단지코드','단지명']].drop_duplicates()

In [21]:
print('NA 개수: ',result3['단지명'].isna().sum())
print('특정된 단지코드 수: ',n-result3['단지명'].isna().sum())
print('전체 단지코드 수: ',n)
print('NA 비율: ', round(result3['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  28
특정된 단지코드 수:  11
전체 단지코드 수:  39
NA 비율:  0.07


- 최종 NA : 28

In [93]:
train[train['단지코드']=='C2132']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,단지명
1896,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,동삼주공2단지
1897,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,동삼주공1단지
1898,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,부산금곡주공4단지
1899,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,덕천주공2단지
1900,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,주공아파트
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,C2132,2428,상가,부산광역시,임대상가,583.40,1,1.0,D,NaN,NaN,0.0,1.0,756.0,359.0,부산금곡주공4단지
2221,C2132,2428,상가,부산광역시,임대상가,583.40,1,1.0,D,NaN,NaN,0.0,1.0,756.0,359.0,덕천주공2단지
2222,C2132,2428,상가,부산광역시,임대상가,583.40,1,1.0,D,NaN,NaN,0.0,1.0,756.0,359.0,주공아파트
2223,C2132,2428,상가,부산광역시,임대상가,583.40,1,1.0,D,NaN,NaN,0.0,1.0,756.0,359.0,모라주공3단지


- 아래 데이터는 하나의 단지코드 별로 두개 이상의 단지명 보유 $\rightarrow$ 잘못 merge된 케이스

In [133]:
result2

,단지코드,단지명
0,C1925,입암주공3단지아파트
2,C1874,쌍용주공7단지1차아파트
4,C1874,공주옥룡주공1차아파트
6,C1874,읍내동 주공아파트
17,C2416,진주평거 주공2단지
...,...,...
885,C1156,NaN
893,C2142,울산송정2
897,C2186,NaN
900,C2186,대구연경 LH천년나무 2단지


In [149]:
r = result.dropna()
r = r.append(result2.dropna())
result2[result2['단지코드']=='C2132']

,단지코드,단지명
409,C2132,NaN
411,C2132,동삼주공2단지
412,C2132,동삼주공1단지
413,C2132,부산금곡주공4단지
414,C2132,덕천주공2단지
416,C2132,주공아파트
417,C2132,모라주공3단지
419,C2132,모라주공아파트1단지


In [24]:
result = result.dropna()
result = result.append(result2.dropna())
result= result.append(result3.dropna())

In [33]:
tmp = pd.DataFrame(result.groupby('단지코드').count()['단지명']).reset_index()
tmp = np.array(tmp[tmp['단지명']>=2]['단지코드'])
tmp

array(['C1022', 'C1039', 'C1068', 'C1085', 'C1109', 'C1175', 'C1206',
       'C1207', 'C1234', 'C1258', 'C1269', 'C1307', 'C1341', 'C1344',
       'C1439', 'C1451', 'C1601', 'C1616', 'C1681', 'C1732', 'C1740',
       'C1744', 'C1790', 'C1859', 'C1874', 'C1875', 'C1894', 'C1965',
       'C1970', 'C2034', 'C2038', 'C2070', 'C2082', 'C2085', 'C2109',
       'C2127', 'C2132', 'C2133', 'C2135', 'C2173', 'C2186', 'C2190',
       'C2227', 'C2232', 'C2258', 'C2289', 'C2310', 'C2325', 'C2352',
       'C2378', 'C2393', 'C2416', 'C2453', 'C2530', 'C2563', 'C2597',
       'C2621', 'C2627', 'C2635', 'C2657'], dtype=object)

In [26]:
train = pd.merge(train,result,how='left',on='단지코드')

In [128]:
result3[result3['단지코드']=='C2132']

,단지코드,단지명
3,C2132,NaN


In [127]:
r[r['단지코드']=='C2132']

,단지코드,단지명
443,C2132,동삼주공2단지
444,C2132,동삼주공1단지
445,C2132,부산금곡주공4단지
446,C2132,덕천주공2단지
447,C2132,주공아파트
448,C2132,모라주공3단지
449,C2132,모라주공아파트1단지


In [126]:
a[a['단지코드']=='C2132']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,단지명
1896,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,동삼주공2단지
1897,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,동삼주공1단지
1898,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,부산금곡주공4단지
1899,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,덕천주공2단지
1900,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1.0,F,3912000.0,67260.0,0.0,1.0,756.0,359.0,주공아파트
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,C2132,2428,상가,부산광역시,임대상가,583.40,1,1.0,D,NaN,NaN,0.0,1.0,756.0,359.0,부산금곡주공4단지
2221,C2132,2428,상가,부산광역시,임대상가,583.40,1,1.0,D,NaN,NaN,0.0,1.0,756.0,359.0,덕천주공2단지
2222,C2132,2428,상가,부산광역시,임대상가,583.40,1,1.0,D,NaN,NaN,0.0,1.0,756.0,359.0,주공아파트
2223,C2132,2428,상가,부산광역시,임대상가,583.40,1,1.0,D,NaN,NaN,0.0,1.0,756.0,359.0,모라주공3단지


In [125]:
result3

,단지코드,단지명
0,C1616,법동 주공아파트3단지
3,C2132,NaN
7,C1899,NaN
9,C1344,엘에이치 서초4단지
11,C1344,NaN
15,C2255,NaN
21,C2352,강남LH3단지
31,C1039,엘에이치 서초 3단지
42,C2659,NaN
53,C2314,NaN


In [49]:
pd.rresult

,단지코드,단지명
0,C2483,포항장량휴먼시아1단지
32,C2515,미수휴먼시아
60,C1407,도안 휴먼시아 4단지
137,C1945,호평마을 주공휴먼시아
155,C1470,장신휴먼시아1단지
...,...,...
173,C1451,옥정천년나무16단지
183,C1258,창원자은3 LH천년나무
252,C2225,김해율하2 A3블록
261,C1732,광주효천1 A-3BL


In [45]:
cnt

20

In [47]:
len(tmp)

60

In [ ]:
train[train['']]

In [48]:
for i in 

,단지코드,단지명
0,C2483,포항장량휴먼시아1단지
32,C2515,미수휴먼시아
60,C1407,도안 휴먼시아 4단지
137,C1945,호평마을 주공휴먼시아
155,C1470,장신휴먼시아1단지
...,...,...
173,C1451,옥정천년나무16단지
183,C1258,창원자은3 LH천년나무
252,C2225,김해율하2 A3블록
261,C1732,광주효천1 A-3BL


In [120]:
r

,단지코드,단지명
0,C2483,포항장량휴먼시아1단지
1,C2515,미수휴먼시아
2,C1407,도안 휴먼시아 4단지
3,C1945,호평마을 주공휴먼시아
4,C1470,장신휴먼시아1단지
...,...,...
533,C1451,옥정천년나무16단지
534,C1258,창원자은3 LH천년나무
535,C2225,김해율하2 A3블록
536,C1732,광주효천1 A-3BL


In [94]:
result[result['단지코드']=='C2132']

,단지코드,단지명
609,C2132,NaN


In [54]:
train[train['임대건물구분']=='상가']['지역'].unique()

array(['강원도', '충청남도', '경상남도', '대전광역시', '부산광역시', '제주특별자치도'], dtype=object)